# Example Pyllelic Use-Case Notebook

## Background

This notebook illustrates the import and use of `pyllelic` in a jupyter environment.

Source code: <https://github.com/Paradoxdruid/pyllelic>

Documentation: <https://paradoxdruid.github.io/pyllelic/>

## Pre-setup / File preparation

### Obtaining fastq data

We can download rrbs (reduced representation bisulfite sequencing) data from the Encode project:
<http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeHaibMethylRrbs/>

Those files are in unaligned fastq format.  We will need to align these to a reference human genome.

### Aligning reads (using process.py)

To align reads, we'll use bowtie2 and samtools (through its pysam wrapper).

First, we need to download a genomic index sequence: <http://hgdownload.soe.ucsc.edu/goldenPath/hg19>

```python
# Processing imports
from pathlib import Path
```

```python
# Set up file paths
index = Path(
    "/home/andrew/allellic/hg19.p13.plusMT.no_alt_analysis_set//hg19.p13.plusMT.no_alt_analysis_set"
)
fastq = Path("/home/andrew/allellic/wgEncodeHaibMethylRrbsU87HaibRawDataRep1.fastq.gz")
```

**WARNING:** The next command is processor, RAM, and time intensive, and only needs to be run once!

```python
# Convert fastq to bam
from pyllelic import process
process.bowtie2_fastq_to_bam(index={bowtie_index_filename_without_suffix},
                                      fastq={fastq_file_name},
                                      cores=6)
```

Notes:

* cores is number of processor cores, adjust for your system
* instead of `out.bam` use a filename that encodes cell-line and tissue.  Our convention is: `fh_CELLLINE_TISSUE.TERT.bam`

Next, we need to sort and index the bam file using samtools functions.

```python
# Sort the bamfile
bamfile = Path("/home/andrew/allellic/wgEncodeHaibMethylRrbsU87HaibRawDataRep1.bam")
process.process_pysam_sort(bamfile)
```

```python
# Create an index of the sorted bamfile
sorted_bam = Path("")
process.process_pysam_index(b)
```

Now, that sorted file (again, rename to capture cell-line and tissue info) is ready to be put in the `test` folder for analysis by pyllelic!

## Set-up

In [ ]:
from pyllelic import pyllelic

1. Set up your disk location:  ```base_path``` should be the directory we'll do our work in
2. Make a sub-directory under ```base_path``` with a folder named ```test``` and put the ```.bam``` and ```.bai``` files in ```test```

In [ ]:
config = pyllelic.set_up_env_variables(
    base_path="/home/andrew/allellic/",  # Windows WSL set-up
    #     base_path="/Users/abonham/documents/test_allelic/",  # OSX setup
    prom_file="TERT-promoter-genomic-sequence.txt",
    prom_start="1293200",
    prom_end="1296000",
    chrom="5",
    offset=1298163,
)

## Main Parsing Functions

### Find files to analyze

In [ ]:
files_set = pyllelic.make_list_of_bam_files(config)

In [ ]:
# files_set # uncomment for debugging

In [ ]:
# files_set = files_set[0:2]  # grab only first twenty files for quick run

In [ ]:
files_set = ["fh_OVK18_OVARY.TERT.bam", 'fh_NCIH2171_LUNG.TERT.bam']

### Perform full methylation analysis and generate data object

**Warning**: This step is processor and time intensive and will perform all processing and analysis of the bam file data.

In [ ]:
data = pyllelic.GenomicPositionData(config=config, files_set=files_set)

### Check main data outcomes

In [ ]:
", ".join(data.positions)

In [ ]:
data.means.head()

In [ ]:
data.modes.head()

In [ ]:
data.diffs.head()

In [ ]:
data.diffs.loc[:,data.diffs.any()]

In [ ]:
data.quma_results[data.means.index[0]].values.head()

## Write Output

### Save entire object as pickle

In [ ]:
# data.save_pickle("test_data2.pickle")

### Save Quma Results to excel

In [ ]:
# data.save("output2.xlsx")

### Save analysis files (means, modes, diffs) to excel

In [ ]:
# data.write_means_modes_diffs("Full2") # Sets the filename stub

### Reopen saved object

In [ ]:
data = pyllelic.GenomicPositionData.from_pickle("test_data2.pickle")

## Initial Data Analysis

### View raw data of methylation percentage per read

In [ ]:
data.heatmap(min_values=1, height=300)

In [ ]:
data.sig_methylation_differences()

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
df = data.individual_data

In [ ]:
def multiple_values(value):
    if isinstance(value, list):
#         print(len(set(value)))
        if len(set(value)) > 1:
            return True
        return False
    return False

In [ ]:
multi = df[df.applymap(multiple_values)]

In [ ]:
multi

In [ ]:
multi.dropna(axis=1, how="all")

### Find values with a methylation difference above a threshold

In [ ]:
import pandas as pd

In [ ]:
DIFF_THRESHOLD = 0.001

In [ ]:
large_diffs = data.diffs[(data.diffs >= DIFF_THRESHOLD).any(1)].dropna(
    how="all", axis=1
)

In [ ]:
large_diffs.head()

In [ ]:
# temp = large_diffs.loc["HUH6"]
# temp.loc[(temp != 0)].dropna()

In [ ]:
interesting = {}
for index, row in large_diffs.iterrows():
    if row.loc[(row != 0)].dropna().any():
        interesting[index] = row.loc[(row != 0)].dropna()

In [ ]:
big_diffs = pd.DataFrame.from_dict(interesting)

In [ ]:
big_diffs.head()

In [ ]:
big_diffs_counts = big_diffs.dropna(axis=1, how="all").count(axis=1).to_frame()

In [ ]:
fig = px.bar(big_diffs_counts, template="seaborn")
fig.update_layout(xaxis_type="linear", showlegend=False, barmode="stack")
fig.update_xaxes(
    tickformat="r", tickangle=45, nticks=40, title="Position", range=[1292981, 1295979]
)
fig.update_yaxes(title="# of large differences")
fig.update_traces(width=50)
fig.show()

In [ ]:
# big_diffs.to_excel("big_diffs.xlsx")

## Visualizing Data

In [ ]:
import plotly.express as px

### Raw quma results

In [ ]:
data.quma_results[data.means.index[0]].values.head()

In [ ]:
data.individual_data.head()

### Histograms of reads at a cell line and genomic position

In [ ]:
CELL = data.means.index[1]
POS = data.means.columns[5]
data.histogram(CELL, POS)

### Plotting read methylation distributions

In [ ]:
import pandas as pd

In [ ]:
df = data.individual_data.copy()

In [ ]:
df.head()

In [ ]:
def to_1D(series):
    """From https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173"""
    if isinstance(series, pd.Series):
        series = series.dropna()
        return pd.Series([x for _list in series for x in _list])

In [ ]:
POS = data.means.columns[5]

In [ ]:
to_1D(df.loc[:,POS]).value_counts()

In [ ]:
to_1D(df.loc[:,POS])

In [ ]:
to_1D(df.loc[:,POS]).mode()

#### Distribution at one cell line, position

In [ ]:
px.bar(to_1D(df.loc[:,POS]).value_counts(normalize=True))

#### Distribution across a cell line

In [ ]:
CELL = data.means.index[1]

In [ ]:
df2 = df.loc[CELL]

In [ ]:
df2.head()

In [ ]:
df2.explode().value_counts()

In [ ]:
px.violin(df2.explode())

#### Distribution across all cell lines

In [ ]:
all_values = []

In [ ]:
for each in df.index:
    temp = pd.Series(df.loc[each].explode())
    #     print(temp)
    all_values.append(temp)
#     print(all_values)

In [ ]:
flat_list = [item for sublist in all_values for item in sublist]

In [ ]:
flat_series = pd.Series(flat_list)

In [ ]:
flat_series.value_counts()

In [ ]:
px.violin(flat_series)

### Identify large differences

In [ ]:
def find_big_diffs(df, min_diff: float):
    out = {}
    for each in df.columns:
        #     print(each)
        mean = to_1D(df[each].dropna()).mean()
        mode = to_1D(df[each].dropna()).mode()
        #     print(mode)
        if not mode.empty:
            #         print(f"Mean: {mean}, mode: {mode}")
            diff = abs(mean - mode.values[0])
            if diff > min_diff:
                out[each] = diff
    #                 print(f"Position: {each}, diff: {diff}")
    #                 print(out)
    if out:
        return out

In [ ]:
big_ones = find_big_diffs(df, 0.1)
big_ones

In [ ]:
big_ones

## Statistical Tests for Normality

In [ ]:
summ = data.summarize_allelic_data()

In [ ]:
summ.head()

In [ ]:
ad_big_diffs = summ.pivot(index="position", columns="cellLine", values="ad_stat")

In [ ]:
ad_big_diffs.head()

In [ ]:
ad_big_diffs = ad_big_diffs.dropna(axis=1, how="all").count(axis=1).to_frame()

In [ ]:
ad_big_diffs.shape

In [ ]:
import plotly.express as px

fig = px.bar(ad_big_diffs, template="seaborn")
fig.update_layout(
    xaxis_type="linear",
    showlegend=False,
    title="TERT Dataset Significant Methylation Differences",
    barmode="overlay",
)
fig.update_xaxes(
    tickformat="r", tickangle=45, nticks=40, title="Position", range=[1292981, 1295979]
)
fig.update_yaxes(title="# of significant differences")
fig.update_traces(width=50)
fig.show()

In [ ]:
ad_big_diffs.values.flatten()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=ad_big_diffs.index, y=ad_big_diffs.values.flatten()))
fig.update_layout(
    xaxis_type="linear",
    showlegend=False,
    title="Significant Methylation Differences",
#     barmode="overlay",
    template="seaborn"
)
fig.update_xaxes(
    tickformat="r", tickangle=45, nticks=40, title="Position", range=[int(ad_big_diffs.index.min()), int(ad_big_diffs.index.max())],
)
fig.update_yaxes(title="# of significant differences")
fig.update_traces(width=50)
fig.show()